In [1]:
import tensorflow as tf
import numpy as np

In [ ]:
import os
import shutil
import glob

def restructure_for_tensorflow(source_dir, output_dir):
    os.makedirs(f"{output_dir}/train", exist_ok=True)
    os.makedirs(f"{output_dir}/test", exist_ok=True)
    
    classes = [d for d in os.listdir(source_dir) if os.path.isdir(os.path.join(source_dir, d))]
    
    for class_name in classes:
        os.makedirs(f"{output_dir}/train/{class_name}", exist_ok=True)
        os.makedirs(f"{output_dir}/test/{class_name}", exist_ok=True)
        
        train_images = glob.glob(f"{source_dir}/{class_name}/train/good/*")
        for img in train_images:
            shutil.copy2(img, f"{output_dir}/train/{class_name}/")
        
        test_images = glob.glob(f"{source_dir}/{class_name}/test/*/*")
        for img in test_images:
            shutil.copy2(img, f"{output_dir}/test/{class_name}/")
        
        print(f"Copied {class_name}: {len(train_images)} train, {len(test_images)} test")

restructure_for_tensorflow(r"C:\Users\DELL\mechanical\mvtec_anomaly_detection", "dataset_tf")


Copied bottle: 209 train, 146 test
Copied cable: 224 train, 210 test
Copied capsule: 219 train, 177 test
Copied carpet: 280 train, 153 test
Copied grid: 264 train, 102 test
Copied hazelnut: 391 train, 145 test
Copied leather: 245 train, 179 test
Copied metal_nut: 220 train, 185 test
Copied pill: 267 train, 275 test
Copied screw: 320 train, 160 test
Copied tile: 230 train, 134 test
Copied toothbrush: 269 train, 72 test
Copied transistor: 213 train, 130 test
Copied wood: 247 train, 98 test
Copied zipper: 240 train, 219 test


In [11]:
# train_dataset = tf.keras.utils.image_dataset_from_directory(
#      r"C:\Users\DELL\mechanical\dataset_tf\train\",
#     labels='inferred', 
#     label_mode='int',  
#     image_size=(224, 224),
#     batch_size=32,
#     subset='training',
#     validation_split=0.2,
#     seed=123
# )

In [28]:
train_dataset = tf.keras.utils.image_dataset_from_directory(
    r"C:\Users\DELL\mechanical\dataset_tf\train\bottle",
    labels=None, 
    image_size=(224, 224),
    batch_size=32
)
train_dataset = train_dataset.map(lambda x: (x, tf.zeros(tf.shape(x)[0], dtype=tf.int32)))

Found 209 files belonging to 1 classes.


In [29]:
# test_dataset = tf.keras.utils.image_dataset_from_directory(
#     r"dataset_tf/test",
#     labels='inferred', 
#     label_mode='int',  
#     image_size=(224, 224),
#     batch_size=32
# )

In [30]:
import tensorflow as tf

test_dataset = tf.keras.utils.image_dataset_from_directory(
    r"C:\Users\DELL\mechanical\dataset_tf\test\bottle",
    labels=None, 
    image_size=(224, 224),
    batch_size=32
)

test_dataset = test_dataset.map(lambda x: (x, tf.zeros(tf.shape(x)[0], dtype=tf.int32)))


Found 146 files belonging to 1 classes.


In [31]:
class_name_train = train_dataset.class_names
class_name_train

AttributeError: 'MapDataset' object has no attribute 'class_names'

In [32]:
class_name_test = test_dataset.class_names
class_name_test

AttributeError: 'MapDataset' object has no attribute 'class_names'

In [33]:
from keras import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras_tuner import RandomSearch

In [34]:
def cnn_model(inp):
    model = Sequential()
    model.add(Conv2D(filters = inp.Int('conv_1_filters', min_value = 32, max_value = 64, step = 6),
                     kernel_size = inp.Choice('conv_1_kernal', values = [3, 5]),
                     activation = 'elu',
                     kernel_initializer='he_normal',
                     input_shape = (224, 224, 3),
                     ))
    model.add(MaxPooling2D(2,2))
    model.add(Flatten())
    model.add(Dense(units = inp.Int('Dense_1', min_value = 32, max_value = 128),
                    activation = 'elu',
                    kernel_initializer = 'he_uniform'))
    model.add(Dropout(0.2)) 
    model.add(Dense(15,activation='softmax'))    
    model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

    return model

In [35]:
rs = RandomSearch(
    cnn_model,
    objective='val_accuracy',
    max_trials=5,
    directory='outputwithdropout03',
    project_name='Mechanical',
    overwrite=True  
)

In [36]:
val_size = 0.2
train_size = int((1 - val_size) * len(train_dataset))

train_ds = train_dataset.take(train_size)
val_ds = train_dataset.skip(train_size)

def preprocess(image, label):
    image = tf.image.resize(image, [224, 224])
    image = image / 255.0  
    return image, label

train_ds = train_ds.map(preprocess)
val_ds = val_ds.map(preprocess)

In [37]:
rs.search(train_ds.unbatch().batch(16), 
          validation_data=val_ds.unbatch().batch(16),
          epochs=3)

Trial 5 Complete [00h 00m 06s]
val_accuracy: 1.0

Best val_accuracy So Far: 1.0
Total elapsed time: 00h 00m 38s
